<a href="https://colab.research.google.com/github/EliorDS/TellMeWhy/blob/main/chocolat_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
files.upload()  # Select the kaggle.json file

In [3]:
import os
import zipfile

# Make a directory for kaggle
os.makedirs("/root/.kaggle", exist_ok=True)

# Move the kaggle.json file to this directory
!mv kaggle.json /root/.kaggle/

# Set permissions (required)
!chmod 600 /root/.kaggle/kaggle.json


In [4]:
!kaggle datasets download -d atharvasoundankar/chocolate-sales

Dataset URL: https://www.kaggle.com/datasets/atharvasoundankar/chocolate-sales
License(s): MIT


In [5]:
with zipfile.ZipFile("chocolate-sales.zip", 'r') as zip_ref:
    zip_ref.extractall("chocolate-sales")

In [6]:
# Import libraries and set up the notebook
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import matplotlib
matplotlib.use('Agg')  # Use Agg backend
import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns
sns.set(style="whitegrid")

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_squared_error

# For permutation importance
from sklearn.inspection import permutation_importance

# Display version info (if needed for debugging)
print("Python version:", pd.__version__)
print("Pandas version:", pd.__version__)


Python version: 2.2.2
Pandas version: 2.2.2


In [10]:
# Load the dataset
file_path = '/content/chocolate-sales/Chocolate Sales.csv'
df = pd.read_csv(file_path)

# Display the first few rows of the dataframe to inspect it (commented out since outputs are not included in this JSON)
print(df.head())

# Convert the 'Date' column to datetime. If an error is encountered, ensure the format is proper.
df['Date'] = pd.to_datetime(df['Date'], errors='coerce')

# Check for missing dates after conversion
# Note: If missing values appear, this is an important step to be aware of for notebook developers.
# print('Missing Date values:', df['Date'].isnull().sum())

# Additional data type conversion or cleaning can be added here if necessary


     Sales Person    Country              Product       Date    Amount  \
0  Jehu Rudeforth         UK      Mint Chip Choco  04-Jan-22   $5,320    
1     Van Tuxwell      India        85% Dark Bars  01-Aug-22   $7,896    
2    Gigi Bohling      India  Peanut Butter Cubes  07-Jul-22   $4,501    
3    Jan Morforth  Australia  Peanut Butter Cubes  27-Apr-22  $12,726    
4  Jehu Rudeforth         UK  Peanut Butter Cubes  24-Feb-22  $13,685    

   Boxes Shipped  
0            180  
1             94  
2             91  
3            342  
4            184  
Missing Date values: 0
